In [ ]:
! pip install konlpy

In [ ]:
!pip install -U imbalanced-learn

In [ ]:
import pandas as pd
from konlpy.tag import Okt;
import pickle

In [ ]:
def tokenizer(text):
  okt = Okt()
  return okt.morphs(text)

In [ ]:
def load_data(path):
  df = pd.read_csv(path)
  df = df[df.label != 2] 

  text_list = df['sentence'].tolist()
  label_list = df['label'].tolist()

  return text_list, label_list

In [ ]:
def split(text_list, label_list):

  from sklearn.model_selection import train_test_split

  text_train, text_test, label_train, label_test = train_test_split(text_list, label_list, test_size=0.2, random_state=42)

  return text_train, text_test, label_train, label_test

In [ ]:
def learn(X_train, X_test, y_train, y_test, model):
  
  # 데이터 단어사전 -> 백터화
  from sklearn.feature_extraction.text import TfidfVectorizer
  tfidf = TfidfVectorizer(lowercase=False, tokenizer=tokenizer)

  #from sklearn.linear_model import LogisticRegression
  #logistic = LogisticRegression(C=10.0, penalty='l2', random_state=0)

  from sklearn.svm import LinearSVC
  svm = LinearSVC()

  from sklearn.pipeline import Pipeline
  pipe = Pipeline([('vect', tfidf), ('clf', svm)])

  pipe.fit(X_train, y_train)

  from sklearn.metrics import accuracy_score
  from sklearn.metrics import classification_report
  y_pred = pipe.predict(X_test)
  print(accuracy_score(y_test, y_pred))
  print(classification_report(y_test, y_pred))

  with open(model, 'wb') as fp:
    pickle.dump(pipe, fp)
  
  print('Model Saved')

In [ ]:
def test(model):

  with open(model, 'rb') as fp:
    pipe = pickle.load(fp)
  
  import numpy as np

  while True:
    text = input('리뷰를 작성해주세요: ')
    if text == 'end':
      break
    str = [text]
    r1 = np.max(pipe.predict_proba(str)*100)
    r2 = pipe.predict(str)[0]
    print('acc =', r1)
    print(r2)

In [ ]:
def tonumpy(text_list, label_list):
  
  import numpy as np
  X_imb, y_imb = np.array(text_list), np.array(label_list)
  X_imb, y_imb = X_imb.reshape((-1, 1)), y_imb.reshape((-1, 1))

  return X_imb, y_imb

In [ ]:
def toList(X_samp):
  a = []
  for i in X_samp:
    a.append(i[0])
  return a

In [ ]:
def under_sampling(text_list, label_list):

  X_imb, y_imb = tonumpy(text_list, label_list)

  from imblearn.under_sampling import RandomUnderSampler
  X_samp, y_samp = RandomUnderSampler(random_state=0).fit_resample(X_imb, y_imb)

  return toList(X_samp), y_samp.tolist()

In [ ]:
def over_sampling(text_list, label_list):
  X_imb, y_imb = tonumpy(text_list, label_list)

  from imblearn.over_sampling import RandomOverSampler
  X_samp, y_samp = RandomOverSampler(random_state=0).fit_resample(X_imb, y_imb)

  return toList(X_samp), y_samp.tolist()

In [ ]:
def train(model, mode):
  t, l = load_data(path)
  if mode == -1:
    t, l = under_sampling(t, l)
  if mode == 1:
    t, l = over_sampling(t, l)
  text_train, text_test, label_train, label_test = split(t, l)
  learn(text_train, text_test, label_train, label_test, model)

In [ ]:
def label_csv(model,c_path):
  df = pd.read_csv(path)
  df = df[df['sentence'].notna()]

  text_list = df['fixed'].tolist()

  with open(model, 'rb') as fp:
    pipe = pickle.load(fp)

  import numpy as np

  p_label = []
  p_proba = []

  for text in text_list:
    str = [text]
    p_proba.append(np.max(pipe.predict_proba(str)*100))
    p_label.append(pipe.predict(str)[0])

  df['label'] = p_label
  df['probability'] = p_proba 

  df.to_csv('spell_check_label.csv')
  print('csv saved')

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/spell_check_label.csv')

In [ ]:
path = '/content/drive/MyDrive/spell_check.csv'
train('none_ori', 0)

0.9399974946761869
              precision    recall  f1-score   support

           0       0.85      0.83      0.84      1485
           1       0.96      0.97      0.96      6498

    accuracy                           0.94      7983
   macro avg       0.90      0.90      0.90      7983
weighted avg       0.94      0.94      0.94      7983

Model Saved


In [ ]:
path = '/content/drive/MyDrive/spell_check_unlabel.csv'

#oversampling mode = 1
train('oversamp.dat', 1)

# 테스트 - end 입력시 종료
#test('/content/drive/MyDrive/models/over.dat')

# 파일 레이블
c_path = '/content/drive/MyDrive/spell_check_unlabel.csv'
label_csv('/content/drive/MyDrive/models/over.dat', c_path)

csv saved
